In [1]:
from scipy.optimize         import newton
from sklearn.neighbors      import KDTree
from scipy.sparse           import csr_matrix
from scipy.sparse.csgraph   import dijkstra
from scipy.linalg           import eig
from sklearn.decomposition  import PCA
import numpy                as np
import gudhi                as gd
import warnings
import gurobipy             as gp
from gurobipy               import GRB
import matplotlib.pyplot    as plt
from kneed                  import KneeLocator
from scipy.spatial          import distance_matrix
from ripser                 import Rips
from sklearn.preprocessing  import PolynomialFeatures
import sympy                as sp
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.problem     import Problem
from pymoo.optimize         import minimize
import tadasets
import rml

%matplotlib qt

In [2]:
n_points = 1500

In [3]:
params = {'max_components':5, 'S':0.5, 'k':10, 'threshold_var':0.05, 'edge_sen':1.5, 'k0':10, 'beta':[0.8, 3, 0.2, 0.2]}
#params = {'max_components':5, 'S':0.5, 'k':10, 'threshold_var':0.05, 'edge_sen':2, 'k0':3, 'beta':None}

b_params = {'k':10, 'threshold_var':0.08, 'edge_sen':1} 

In [4]:
sphere2 = tadasets.dsphere(n=int(n_points*2))
sphere2 = sphere2[sphere2[:, 2]>=0]

sphere5 = tadasets.dsphere(n=int(n_points*1.3))
sphere5 = sphere5[sphere5[:, 2]>=-0.7]

In [5]:
S = rml.Simplex()
S.build_simplex(sphere5, **params)
_ = S.normal_coords_trade(**params)

Restricted license - for non-production use only - expires 2023-10-25
535
74


In [6]:
def local_neighborhood(S, scale):
    '''
    Computes an annular neighborhood for every point in the point cloud.
    
    data: array describing the point cloud
    scale: list [k1, k2] with k1, k2 integers describing the number of nearest neighbors that will comprise the annulus
    '''

    D = dijkstra(S.edge_matrix)  # go from pointcloud manifold to projection

    S_b = rml.Simplex()
    S_b.build_simplex(S.coords, **b_params)
    D_coords = dijkstra(S_b.edge_matrix)
    
    n = scale[1]-scale[0]
    m = len(D)

    local_neigh = np.ndarray(shape=(m,n), dtype=int, order='F')
    radius = np.ndarray(shape=(m,2), dtype=float, order='F')
    radius_coords = np.ndarray(shape=(m,2), dtype=float, order='F')

    for i in range(m):
        local_neigh[i] = np.argsort(D[i])[scale[0]:scale[1]] # the annulus neighborhood of point i in data

    D.sort()
    D_coords.sort()

    for i in range(m):
        radius[i] = [D[i][scale[0]], D[i][scale[1]]] # the pair [r1,r2] of radii associated to the annulus neighborhood
        radius_coords[i] = [D_coords[i][scale[0]], D_coords[i][scale[1]]]

    return local_neigh, radius, radius_coords 

In [ ]:
def compute_local_persistence(data, scale, d):
    '''
    Classify every point in the point cloud depending its local homology at degree d-1
    
    data: array describing the point cloud
    scale: list [k1, k2] with k1, k2 integers describing the number of nearest neighbors that will comprise the annulus
    d: the estimated intrinsic dimension of the point cloud
    '''
    k1 = scale[0]
    k2 = scale[1]
    
    local_neigh, radius, radius_coords = local_neighborhood(data, [k1, k2])
    
    rips = Rips(maxdim = d-1)
    mask = []
    for i in range(len(data)):
        dgm = rips.fit_transform(data[local_neigh[i]])
        
        # here we only focus on betti d-1

        lifetime = dgm[d-1][:,1]-dgm[d-1][:,0]

        r1 = radius[i][0]
        r2 = radius[i][1]
            
        N = np.where(lifetime>r2-r1)[0]

        if len(N)==0:
            mask.append(0) # boundary
        elif len(N)==1:
            mask.append(1) # regular point
        else:
            mask.append(2) # singular point

    return np.array(mask)

In [10]:
def show_boundary(S0, **kwargs):
    """
    
    """

    mask = compute_local_persistence(S0, [40, 80], S0.dim)
    boundary_points = np.where(mask==0)[0]
    non_boundary_points = np.where(mask!=0)[0]

    fig = plt.figure(figsize=(20, 10))
    fig.suptitle(", ".join([i+"="+str(kwargs[i]) for i in kwargs.keys()]) + f', dim={S0.dim}, n={len(S0.pointcloud)}')

    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    ax2 = fig.add_subplot(1, 2, 2)

    ax1.scatter3D(S0.pointcloud[non_boundary_points, 0], S0.pointcloud[non_boundary_points, 1], S0.pointcloud[non_boundary_points, 2], c=S0.pointcloud[non_boundary_points, 0], alpha=0.7) 
    ax1.scatter3D(S0.pointcloud[boundary_points, 0], S0.pointcloud[boundary_points, 1], S0.pointcloud[boundary_points, 2], c='r')

    for i in range(len(S0.pointcloud)):
        for k in S0.edges[i]:
            ax1.plot3D([S0.pointcloud[i][0], S0.pointcloud[k][0]],[S0.pointcloud[i][1], S0.pointcloud[k][1]], [S0.pointcloud[i][2], S0.pointcloud[k][2]], color='black', alpha=0.1)

    ax2.scatter(S0.coords[non_boundary_points, 0], S0.coords[non_boundary_points, 1], c=S0.pointcloud[non_boundary_points, 0], alpha=0.7)
    ax2.scatter(S0.coords[boundary_points, 0], S0.coords[boundary_points, 1], c='r')

    plt.show()

In [11]:
def show_pointcloud_boundary(S0, show_coords=False, **kwargs):

    mask = rml.compute_local_persistence(S0.pointcloud, [40, 80], S0.dim) 
    boundary_points = np.where(mask==0)[0]
    non_boundary_points = np.where(mask!=0)[0]

    fig = plt.figure(figsize=(20, 10))
    fig.suptitle(", ".join([i+"="+str(kwargs[i]) for i in kwargs.keys()]) + f', dim={S0.dim}, n={len(S0.pointcloud)}')

    ax1 = fig.add_subplot(1, 1, 1, projection='3d') if not show_coords else fig.add_subplot(1, 2, 1, projection='3d')
    if show_coords:
        ax2 = fig.add_subplot(1, 2, 2)


    ax1.scatter3D(S0.pointcloud[non_boundary_points, 0], S0.pointcloud[non_boundary_points, 1], S0.pointcloud[non_boundary_points, 2], c=S0.pointcloud[non_boundary_points, 0], alpha=0.7) 
    ax1.scatter3D(S0.pointcloud[boundary_points, 0], S0.pointcloud[boundary_points, 1], S0.pointcloud[boundary_points, 2], c='r')

    for i in range(len(S0.pointcloud)):
        for k in S0.edges[i]:
            ax1.plot3D([S0.pointcloud[i][0], S0.pointcloud[k][0]],[S0.pointcloud[i][1], S0.pointcloud[k][1]], [S0.pointcloud[i][2], S0.pointcloud[k][2]], color='black', alpha=0.1)
    
    if show_coords:
        ax2.scatter(S0.coords[non_boundary_points, 0], S0.coords[non_boundary_points, 1], c=S0.pointcloud[non_boundary_points, 0], alpha=0.7)
        ax2.scatter(S0.coords[boundary_points, 0], S0.coords[boundary_points, 1], c='r')
    
    plt.show()

In [14]:
show_boundary(S)

Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


In [12]:
show_pointcloud_boundary(S, show_coords=True, **params)

Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


In [43]:
def compute_boundary(S0, **kwargs):
    """
    
    """
    mask = rml.compute_local_persistence(S0.coords, [40, 80], S0.dim)  # test more parameters
    dist_matrix, _ = dijkstra(S0.edge_matrix, return_predecessors=True) 
    boundary_points = np.where(mask==0)[0]

    p_idx = boundary_points[0]
    p_dist = dist_matrix[p_idx, boundary_points]

    S_b = rml.Simplex()
    S_b.build_simplex(S0.coords, **kwargs)

    fig = plt.figure(figsize=(20, 10))

    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)

    ax1.scatter(S_b.pointcloud[:, 0], S_b.pointcloud[:, 1])
    ax1.scatter(S_b.pointcloud[boundary_points, 0], S_b.pointcloud[boundary_points, 1], color='r')

    for i in range(len(S_b.pointcloud)):
        for k in S_b.edges[i]:
            ax1.plot([S_b.pointcloud[i][0], S_b.pointcloud[k][0]],[S_b.pointcloud[i][1], S_b.pointcloud[k][1]], color='black', alpha=0.1)

    new_boundary = []

    for i in boundary_points:
        edge = S_b.edges[i]
        count = 0
        for j in edge:
            if j in boundary_points:
                count += 1
        if count > len(edge) - count:
            new_boundary.append(i)

    new_boundary = np.asarray(new_boundary)

    ax2.scatter(S_b.pointcloud[:, 0], S_b.pointcloud[:, 1])
    ax2.scatter(S_b.pointcloud[new_boundary, 0], S_b.pointcloud[new_boundary, 1], color='r')

    for i in range(len(S_b.pointcloud)):
        for k in S_b.edges[i]:
            ax2.plot([S_b.pointcloud[i][0], S_b.pointcloud[k][0]],[S_b.pointcloud[i][1], S_b.pointcloud[k][1]], color='black', alpha=0.1)